In [1]:
import os
import sys

import pandas as pd
import numpy as np

In [37]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import RidgeClassifierCV
from sklearn.metrics import roc_auc_score

from sklearn.model_selection import train_test_split

In [18]:
X_test_raw = pd.read_csv('test_set_features.csv')

X_train_raw = pd.read_csv('training_set_features.csv')
y_train_raw = pd.read_csv('training_set_labels.csv')

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.33)

In [3]:
X_train.head()

,respondent_id,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,...,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation
0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,Below Poverty,Not Married,Own,Not in Labor Force,oxchjgsf,Non-MSA,0.0,0.0,NaN,NaN
1,1,3.0,2.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,...,Below Poverty,Not Married,Rent,Employed,bhuqouqj,"MSA, Not Principle City",0.0,0.0,pxcmvdjn,xgwztkwe
2,2,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,"<= $75,000, Above Poverty",Not Married,Own,Employed,qufhixun,"MSA, Not Principle City",2.0,0.0,rucpziij,xtkaffoo
3,3,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,...,Below Poverty,Not Married,Rent,Not in Labor Force,lrircsnp,"MSA, Principle City",0.0,0.0,NaN,NaN
4,4,2.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,...,"<= $75,000, Above Poverty",Married,Own,Employed,qufhixun,"MSA, Not Principle City",1.0,0.0,wxleyezf,emcorrxb


In [4]:
y_train

,respondent_id,h1n1_vaccine,seasonal_vaccine
0,0,0,0
1,1,0,1
2,2,0,0
3,3,0,1
4,4,0,0
...,...,...,...
26702,26702,0,0
26703,26703,0,0
26704,26704,0,1
26705,26705,0,0


In [ ]:
# create two diff predictors for h1n1 and flu?

In [ ]:
# important columns: h1n1_concern, h1n1_knowledge, 

In [ ]:
# interger encode: education, income_poverity
# one hot encode: age_group, race, sex
                # marital status, rent_or_own, employment_status
                # hhs_geo_region, census_msa, employment_industry,
                # employment_occupation

In [ ]:
# column transformer
    # integer encode ordinal
    # one hot encode nominal
# build pipeline to predict
# try out diff predictors

In [6]:
X_train.columns

Index(['respondent_id', 'h1n1_concern', 'h1n1_knowledge',
       'behavioral_antiviral_meds', 'behavioral_avoidance',
       'behavioral_face_mask', 'behavioral_wash_hands',
       'behavioral_large_gatherings', 'behavioral_outside_home',
       'behavioral_touch_face', 'doctor_recc_h1n1', 'doctor_recc_seasonal',
       'chronic_med_condition', 'child_under_6_months', 'health_worker',
       'health_insurance', 'opinion_h1n1_vacc_effective', 'opinion_h1n1_risk',
       'opinion_h1n1_sick_from_vacc', 'opinion_seas_vacc_effective',
       'opinion_seas_risk', 'opinion_seas_sick_from_vacc', 'age_group',
       'education', 'race', 'sex', 'income_poverty', 'marital_status',
       'rent_or_own', 'employment_status', 'hhs_geo_region', 'census_msa',
       'household_adults', 'household_children', 'employment_industry',
       'employment_occupation'],
      dtype='object')

In [ ]:
# try out simple linear classifier using the cols 
# that are already transformed

# h1n1_concern, h1n1_knowledge, behavioral_avoidance, behavioral_face_mask,
# behavioral_wash_hands, behavioral_large_gatherings, behavioral_outside_home, 
# behavioral_touch_face, doctor_recc_h1n1

# 'h1n1_concern', 'h1n1_knowledge',
# 'behavioral_antiviral_meds', 'behavioral_avoidance',
# 'behavioral_face_mask', 'behavioral_wash_hands',
# 'behavioral_large_gatherings', 'behavioral_outside_home',
# 'behavioral_touch_face', 'doctor_recc_h1n1', 'doctor_recc_seasonal',
# 'chronic_med_condition', 'child_under_6_months', 'health_worker',
# 'health_insurance', 'opinion_h1n1_vacc_effective', 'opinion_h1n1_risk',
# 'opinion_h1n1_sick_from_vacc', 'opinion_seas_vacc_effective',
# 'opinion_seas_risk', 'opinion_seas_sick_from_vacc'


In [ ]:
# drop na, join on id, 

In [48]:
# get wanted cols and drop na
X_base_train_cols = X_train.iloc[:, 0:22]
X_base_train_cols = X_base_train_cols.dropna()
X_base_test_cols = X_test.iloc[:,0:22].dropna()

# merge
full_dropped_train = X_base_train_cols.merge(y_train, how='left', on='respondent_id')
X_base_train = full_dropped_train.iloc[:, 1:-2]
y_base_train = full_dropped_train.iloc[:, -2]

full_dropped_test = X_base_test_cols.merge(y_test, how='left', on='respondent_id')
X_base_test = full_dropped_test.iloc[:, 1:-2]
y_base_test = full_dropped_test.iloc[:, -2]



base_fit = RidgeClassifierCV().fit(X_base_train, y_base_train)

In [50]:
base_preds = base_fit.decision_function(X_base_test)

In [51]:
roc_auc_score(y_base_test, base_preds)

0.846780021378816

In [ ]:
# todo: find best features with correlation,
        # maybe find best for h1n1 and one for flu